## Advanced SQL Concepts

#### Schema Reference (Retail ERD)

Assumed available tables in `retail_db`:

- `departments(department_id, department_name)`

- `categories(category_id, category_department_id, category_name)`

- `products(product_id, product_category_id, product_name, product_description, product_price, product_image)`

- `customers(customer_id, customer_fname, customer_lname, customer_email, ...)`

- `orders(order_id, order_date, order_customer_id, order_status)`

- `order_items(order_item_id, order_item_order_id, order_item_product_id, order_item_quantity, order_item_subtotal, order_item_product_price)`

![ERD Placeholder](../images/ecommerce_erd.png)


### CASE Statement for Order Status Classification

The business needs a **standardized, high-level classification** that:
- Clearly indicates whether an order needs attention
- Flags risky transactions early
- Simplifies dashboards and operational monitoring

---

| Order Status         | Meaning                                    | Action Category        |
|----------------------|--------------------------------------------|------------------------|
| COMPLETE             | Order fulfilled successfully               | No Action Needed       |
| CLOSED               | Order closed after fulfillment or settlement | No Action Needed       |
| PENDING_PAYMENT      | Payment not yet received                   | Action Needed          |
| PROCESSING           | Order being processed                      | Action Needed          |
| PAYMENT_REVIEW       | Payment requires verification              | Action Needed          |
| PENDING              | Awaiting next workflow step                | Action Needed          |
| ON_HOLD              | Temporarily paused                         | Action Needed          |
| SUSPECTED_FRAUD      | Potential fraud detected                   | Risky                  |
| CANCELED             | Order canceled                             | Closed / No Action     |

---


```sql
SELECT
    order_id,
    order_date,
    order_status,
    CASE
        WHEN order_status IN ('COMPLETE', 'CLOSED') 
            THEN 'No Action Needed'

        WHEN order_status IN (
            'PENDING_PAYMENT',
            'PROCESSING',
            'PAYMENT_REVIEW',
            'PENDING',
            'ON_HOLD'
        )
            THEN 'Action Needed'

        WHEN order_status = 'SUSPECTED_FRAUD'
            THEN 'Risky'

        WHEN order_status = 'CANCELED'
            THEN 'Closed / No Action'

        ELSE 'Unknown / Review Required'
    END AS order_status_category
FROM orders
ORDER BY order_date;
```

### Common Table Expressions (CTE)

A **Common Table Expression (CTE)** is a temporary, named result set defined using the `WITH` clause.  
It exists only for the duration of the query and helps make complex SQL **more readable, modular, and maintainable**.

CTEs are commonly used for:
- Breaking complex queries into logical steps
- Reusing intermediate results
- Improving query clarity (especially with joins and aggregations)

```sql
WITH orders_needing_action AS (
    SELECT
        order_id,
        order_status
    FROM orders
    WHERE order_status IN (
        'PENDING_PAYMENT',
        'PROCESSING',
        'PAYMENT_REVIEW',
        'ON_HOLD'
    )
)
SELECT *
FROM orders_needing_action;


### Second Highest Salary (Using Subquery)

```sql
SELECT MAX(sal) AS second_highest_salary
FROM emp
WHERE sal < (
    SELECT MAX(sal)
    FROM emp
);

### Second Highest Salary (Using CTE)

```sql
WITH max_salary AS (
    SELECT MAX(sal) AS highest_salary
    FROM emp
)
SELECT MAX(sal) AS second_highest_salary
FROM emp
WHERE sal < (
    SELECT highest_salary
    FROM max_salary
);
